# Libraries

In [26]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn import metrics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [27]:
%run Data_preprocessing.ipynb

************************************************************
There are 6 set of X
X_*_nozero
------------------------------------------------------------
Normalized version
X_*_norm
------------------------------------------------------------
There are 4 set of Y
y_train_t1, y_train_t2, y_valid_t1, y_valid_t2
when training, please use: 'y_train_t1_value,y_train_t2_value,y_valid_t1_value,y_valid_t2_value'
************************************************************


In [28]:
print(X_train_norm.shape)
print(X_valid_norm.shape)
print(X_test_norm.shape)

(16760, 61)
(2394, 61)
(4790, 61)


# Neural Network Using Sklearn build in function

In [41]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(hidden_layer_sizes=(100,),
                   activation='relu',
                   solver='adam', 
                   alpha=0.0001, 
                   batch_size='auto', 
                   learning_rate='constant', 
                   learning_rate_init=0.001,
                   max_iter=300,  
                   random_state=3612, 
                   momentum=0.9, 
                   early_stopping=False, 
                   validation_fraction=0.1)

clf.fit(X_train_norm, y_train_t2_value)

clf.score(X_valid_norm, y_valid_t2_value)

-0.12806230444436162

In [42]:
y_pred = clf.predict(X_valid_norm)

rmse(y_pred, y_valid_t2_value)

2.1181483020077145

# Evaluation Matrix

In [31]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# Hyperparameter and setting tuning

## layers

In [ ]:
layer_list = []
fpr_lst=[]
tpr_lst=[]
roc_auc_lst=[]
for i in range(1,20,5):
    for j in range(1,20,5):
        layer_list.append((i,j))
        
for i in layer_list:
    clf = MLPRegressor(solver="lbfgs",
                        alpha=1e-5,
                        hidden_layer_sizes=i,
                        random_state=1,
                        max_iter=200)

    clf.fit(X_train_norm, y_train_t2_value)
    clf.score(X_valid_norm, y_valid_t2_value)
    y_pred = clf.predict(X_valid_norm)
    fpr, tpr, _ = metrics.roc_curve(y_valid_t2_value, y_pred)
    roc_auc = metrics.roc_auc_score(y_valid_t2_value, y_pred)
    
    fpr_lst.append(fpr)
    tpr_lst.append(tpr)
    roc_auc_lst.append(roc_auc)
    
    
index = roc_auc_lst.index(max(roc_auc_lst))
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k-')
plt.plot(fpr_lst[index], tpr_lst[index], label='FN(area = {:.3f})'.format(max(roc_auc_lst)))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title(f"Layer {layer_list[index]} - ROC curve")
plt.legend(loc="best")
plt.show()

In [ ]:
best_layernumber = layer_list[index]
best_layernumber

## Solver

In [ ]:
solver_list = ["lbfgs","sgd","adam"]
fpr_lst=[]
tpr_lst=[]
roc_auc_lst=[]

for i in solver_list:
    clf = MLPRegressor(solver=i,
                        alpha=1e-5,
                        hidden_layer_sizes=best_layernumber,
                        random_state=1,
                        max_iter=200)

    clf.fit(X_train_norm, y_train_t2_value)
    clf.score(X_valid_norm, y_valid_t2_value)
    y_pred = clf.predict(X_valid_norm)
    fpr, tpr, _ = metrics.roc_curve(y_valid_t2_value, y_pred)
    roc_auc = metrics.roc_auc_score(y_valid_t2_value, y_pred)
    
    fpr_lst.append(fpr)
    tpr_lst.append(tpr)
    roc_auc_lst.append(roc_auc)
    
    
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True)
plt.figure(figsize=(5,5))
for j in range(3):
    plt.figure(1)
    axes[j].plot([0, 1], [0, 1], 'k-')
    axes[j].plot(fpr_lst[j], tpr_lst[j], label='FN(area = {:.3f})'.format(roc_auc_lst[j]))
    axes[j].set_xlabel('False positive rate')
    axes[j].set_ylabel('True positive rate')
    axes[j].set_title(f"{solver_list[j]} - ROC curve")
    axes[j].legend(loc="best")
plt.show()

In [ ]:
index = roc_auc_lst.index(max(roc_auc_lst))
best_solver = solver_list[index]
best_solver

## Learning rate tuning

In [ ]:
fpr_lst=[]
tpr_lst=[]
roc_auc_lst=[]
lr_list = ["constant", "invscaling", "adaptive"]

for i in lr_list:
    clf = MLPRegressor(solver=best_solver,
                        alpha=1e-5,
                        hidden_layer_sizes=best_layernumber,
                        learning_rate=i,
                        random_state=3612,
                        max_iter=200)

    clf.fit(X_train_norm, y_train_t2_value)
    clf.score(X_valid_norm, y_valid_t2_value)
    y_pred = clf.predict(X_valid_norm)
    fpr, tpr, _ = metrics.roc_curve(y_valid_t2_value, y_pred)
    roc_auc = metrics.roc_auc_score(y_valid_t2_value, y_pred)
    
    fpr_lst.append(fpr)
    tpr_lst.append(tpr)
    roc_auc_lst.append(roc_auc)
    
    
index = roc_auc_lst.index(max(roc_auc_lst))
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k-')
plt.plot(fpr_lst[index], tpr_lst[index], label='FN(area = {:.3f})'.format(max(roc_auc_lst)))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title(f"Learning Rate {lr_list[index]} - ROC curve")
plt.legend(loc="best")
plt.show()

In [ ]:
index = roc_auc_lst.index(max(roc_auc_lst))
best_lr_choice =  lr_list[index]
best_lr_choice

# Test

In [ ]:
# Final model
clf1 = MLPClassifier(solver=best_solver,
                    alpha=1e-5,
                    hidden_layer_sizes=best_layernumber,
                    learning_rate=best_lr_choice,
                    random_state=3612,
                    max_iter=200)

clf1.fit(X_train_norm, y_train_t2_value)

clf1.score(X_valid_norm, y_valid_t2_value_2)
y_pred = clf1.predict(X_valid_norm)
fpr, tpr, _ = metrics.roc_curve(y_valid_t2_value, y_pred)
roc_auc = metrics.roc_auc_score(y_valid_t2_value, y_pred)



plt.figure(1)
plt.plot([0, 1], [0, 1], 'k-')
plt.plot(fpr, tpr, label='FN(area = {:.3f})'.format(roc_auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title("ROC curve")
plt.legend(loc="best")
plt.show()

In [ ]:
X_test_norm

In [ ]:

result = clf1.predict(X_test_norm)
res1 = pd.DataFrame(result.tolist(), columns=["Predicted"])
res1.index = X_test.index
res1.index.name = 'Id'
pd.DataFrame(res1).to_csv('out.csv')
#print(torch.count_nonzero(torch.from_numpy(np.array(res1)).to(torch.float32)))
res1.sum(axis=0)